In [17]:
from pulp import *
# initialize the model
prob = LpProblem("multiSteelProd", LpMinimize)

prod_flow = {
             0 :  1800,
             1 :  2400,
             2 :  2200,
             3 :  2700,
             4 :  2500,
             }
cost = {
             0 :  0,
             1 :  7400,
             2 :  7500,
             3 :  7600,
             4 :  7650,
             }

inventory_cost = {
             0 :  0,
             1 :  120,
             2 :  120,
             3 :  120,
             4 :  0,
             }

extra_produce_cost = 50
low_produce_cost = 30

prod_limit= 4000

T = len(prod_flow)
# ---------------------
# VARIABLES
# ---------------------

produce = LpVariable.dicts("produce", range(0, T), 0, None)
carry = LpVariable.dicts("carry", range(0, T), 0, None) 
incr = LpVariable.dicts("incr", range(0, T), 0, None)
decr = LpVariable.dicts("decr", range(0, T ), 0, None) 

# Objective function
prob += sum([cost[i] * produce[i] + inventory_cost[i] * carry[i] + extra_produce_cost * incr[i] + \
             low_produce_cost *decr[i]  for i in range(1,T)]), "Objective"

# Constraints
prob += carry[0] == 0
prob += carry[T-1] == 1500
prob += produce[0] == 1800
prob += decr[0] == 0

prob += produce[1] + 1000 - carry[1] == 2400

for i in range(2, T):
    prob += produce[i] + carry[i - 1] - carry[i] == prod_flow[i]

for i in range(1, T):
    prob += produce[i] - produce[i - 1] == incr[i] - decr[i]

prob.writeLP("multiSteelProd.lp")
    
status = prob.solve(GLPK(options=["--ranges","multiSteelProd.sen"]))
print(status)

#print the result
print("prod")
for i in range(1, T):
    print(produce[i].value())


print("incr")       
for i in range(1, T):
    print(incr[i].value())

print("decr")   
for i in range(1, T):
    print(decr[i].value())
    
print("carry")    
for i in range(1, T):
    print(carry[i].value())


print("Objective", value(prob.objective))

1
prod
1800.0
1800.0
2700.0
4000.0
incr
0.0
0.0
900.0
1300.0
decr
0.0
0.0
0.0
0.0
carry
400.0
0.0
0.0
1500.0
Objective 78098000.0


In [19]:
prob

multiSteelProd:
MINIMIZE
120*carry_1 + 120*carry_2 + 120*carry_3 + 30*decr_1 + 30*decr_2 + 30*decr_3 + 30*decr_4 + 50*incr_1 + 50*incr_2 + 50*incr_3 + 50*incr_4 + 7400*produce_1 + 7500*produce_2 + 7600*produce_3 + 7650*produce_4 + 0
SUBJECT TO
_C1: carry_0 = 0

_C2: carry_4 = 1500

_C3: produce_0 = 1800

_C4: decr_0 = 0

_C5: - carry_1 + produce_1 = 1400

_C6: carry_1 - carry_2 + produce_2 = 2200

_C7: carry_2 - carry_3 + produce_3 = 2700

_C8: carry_3 - carry_4 + produce_4 = 2500

_C9: decr_1 - incr_1 - produce_0 + produce_1 = 0

_C10: decr_2 - incr_2 - produce_1 + produce_2 = 0

_C11: decr_3 - incr_3 - produce_2 + produce_3 = 0

_C12: decr_4 - incr_4 - produce_3 + produce_4 = 0

VARIABLES
carry_0 Continuous
carry_1 Continuous
carry_2 Continuous
carry_3 Continuous
carry_4 Continuous
decr_0 Continuous
decr_1 Continuous
decr_2 Continuous
decr_3 Continuous
decr_4 Continuous
incr_1 Continuous
incr_2 Continuous
incr_3 Continuous
incr_4 Continuous
produce_0 Continuous
produce_1 Continuous
p

In [ ]:
# %load multiSteelProd.sen
GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  Objective = 78098000 (MINimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 _C1          NS        .             .             .              .               -Inf    7.8098e+07 carry_0
                                            .             .                +Inf          +Inf    7.8098e+07

     2 _C10         NS        .             .             .              .               -Inf    7.8098e+07 carry_2
                                         -30.00000        .           450.00000          +Inf   7.80845e+07 incr_3

     3 _C11         NS        .             .             .                -Inf          -Inf          +Inf
                                         -50.00000        .           900.00000          +Inf    7.8053e+07 incr_3

     4 _C12         NS        .             .             .                -Inf          -Inf          +Inf
                                         -50.00000        .          1300.00000          +Inf    7.8033e+07 incr_4

     5 _C2          NS    1500.00000        .         1500.00000      200.00000          -Inf    6.8088e+07 incr_4
                                        7700.00000    1500.00000           +Inf          +Inf          +Inf

     6 _C3          NS    1800.00000        .         1800.00000     1800.00000          -Inf    7.8098e+07 carry_2
                                          10.00000    1800.00000     2100.00000          +Inf    7.8101e+07 incr_3

     7 _C4          NS        .             .             .              .               -Inf    7.8098e+07 decr_0
                                            .             .                +Inf          +Inf    7.8098e+07

     8 _C5          NS    1400.00000        .         1400.00000      500.00000          -Inf    7.1474e+07 incr_3
                                        7360.00000    1400.00000     1400.00000          +Inf    7.8098e+07 carry_2

     9 _C6          NS    2200.00000        .         2200.00000     1300.00000          -Inf    7.1366e+07 incr_3
                                        7480.00000    2200.00000     2200.00000          +Inf    7.8098e+07 carry_2

    10 _C7          NS    2700.00000        .         2700.00000     1800.00000          -Inf    7.1258e+07 incr_3
                                        7600.00000    2700.00000     4000.00000          +Inf    8.7978e+07 incr_4

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   2

Problem:    
Objective:  Objective = 78098000 (MINimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
    11 _C8          NS    2500.00000        .         2500.00000     1200.00000          -Inf    6.8088e+07 incr_4
                                        7700.00000    2500.00000           +Inf          +Inf          +Inf

    12 _C9          NS        .             .             .              .               -Inf    7.8098e+07 carry_2
                                          10.00000        .           300.00000          +Inf    7.8101e+07 incr_3

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   3

Problem:    
Objective:  Objective = 78098000 (MINimum)

   No. Column name  St      Activity      Obj coef   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 carry_1      BS     400.00000     120.00000        .           700.00000      60.00000    7.8074e+07 incr_1
                                            .               +Inf      400.00000     140.00000    7.8106e+07 decr_1

     2 carry_2      BS        .          120.00000        .           450.00000     100.00000    7.8098e+07 incr_2
                                            .               +Inf     -800.00000     130.00000    7.8098e+07 decr_1

     3 carry_3      NL        .          120.00000        .          -900.00000     100.00000     7.808e+07 incr_3
                                          20.00000          +Inf      650.00000          +Inf    7.8111e+07 incr_4

     4 decr_1       NL        .           30.00000        .          -300.00000      10.00000    7.8092e+07 incr_3
                                          20.00000          +Inf         .               +Inf    7.8098e+07 carry_2

     5 decr_2       NL        .           30.00000        .          -450.00000     -30.00000    7.8071e+07 incr_3
                                          60.00000          +Inf         .               +Inf    7.8098e+07 carry_2

     6 decr_3       NL        .           30.00000        .          -900.00000     -50.00000    7.8026e+07 incr_3
                                          80.00000          +Inf           +Inf          +Inf          +Inf

     7 decr_4       NL        .           30.00000        .         -1300.00000     -50.00000    7.7994e+07 incr_4
                                          80.00000          +Inf           +Inf          +Inf          +Inf

     8 incr_1       NL        .           50.00000        .              .          -10.00000    7.8098e+07 carry_2
                                          60.00000          +Inf      300.00000          +Inf    7.8116e+07 incr_3

     9 incr_2       NL        .           50.00000        .              .           30.00000    7.8098e+07 carry_2
                                          20.00000          +Inf      450.00000          +Inf    7.8107e+07 incr_3

    10 incr_3       BS     900.00000      50.00000        .           900.00000      43.33333    7.8092e+07 decr_1
                                            .               +Inf    -4500.00000      60.00000    7.8107e+07 incr_2

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   4

Problem:    
Objective:  Objective = 78098000 (MINimum)

   No. Column name  St      Activity      Obj coef   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
    11 incr_4       BS    1300.00000      50.00000        .          1750.00000      30.00000    7.8072e+07 incr_2
                                            .               +Inf    -6700.00000      60.00000    7.8111e+07 carry_3

    12 produce_1    BS    1800.00000    7400.00000        .          2100.00000    7340.00000     7.799e+07 incr_1
                                            .               +Inf     1800.00000    7420.00000    7.8134e+07 decr_1

    13 produce_2    BS    1800.00000    7500.00000        .          2250.00000    7480.00000    7.8062e+07 incr_2
                                            .               +Inf     1800.00000    7520.00000    7.8134e+07 decr_1

    14 produce_3    BS    2700.00000    7600.00000        .          2700.00000    7590.00000    7.8071e+07 decr_1
                                            .               +Inf     2250.00000    7620.00000    7.8152e+07 incr_2

    15 produce_4    BS    4000.00000    7650.00000        .          4000.00000          -Inf          -Inf
                                            .               +Inf     3350.00000    7670.00000    7.8178e+07 carry_3

    16 carry_0      BS        .             .             .              .               -Inf    7.8098e+07
                                            .               +Inf         .               +Inf    7.8098e+07

    17 carry_4      BS    1500.00000        .             .          1500.00000          -Inf          -Inf
                                            .               +Inf     1500.00000          +Inf          +Inf

    18 produce_0    BS    1800.00000        .             .          1800.00000          -Inf          -Inf
                                            .               +Inf     1800.00000          +Inf          +Inf

    19 decr_0       BS        .             .             .              .               -Inf    7.8098e+07
                                            .               +Inf         .               +Inf    7.8098e+07

End of report
